In [1]:
%load_ext autoreload
from nb_config import *

In [2]:
def simulate(current_time, start_time, end_time, agents, receiver_agents, agents_delay_map, 
             contracts, delay_order, delay_notification):
    start_simulation = time.time()
    
    market_input_ports = ['in_agent_regulator', 'in_agent_journal']
    market_output_ports = ['out_next_journal_agent', 'out_notify_order_journal_agent', 'out_next_regulator_agent']
    orderbooks_map = {
        contract: OrderbookVersion1('ob_' + contract, contract, delay_order, delay_notification)
        for contract in contracts}
    market = Version2Market('market', current_time, float('inf'), orderbooks_map.values(),
                            market_input_ports, market_output_ports, agents_delay_map,
                            start_time=start_time, end_time=end_time)
    connections = [((agent.identifier, 'out_order'), (market.identifier, 'in_agent_regulator'))
                   for agent in agents]
    # Reactive agent observes the output from journal
    connections += [((market.identifier, 'out_next_journal_agent'), (agent.identifier, 'in_next'))
                    for agent in receiver_agents]
    connections += [((market.identifier, 'out_notify_order_journal_agent'), (agent.identifier, 'in_notify_order'))
                    for agent in receiver_agents]
    connections += [((market.identifier, 'out_next_regulator_agent'), (agent.identifier, 'in_notify_order'))
                    for agent in receiver_agents]
    m = SimpleSystem(market=market, agents=agents, connections=connections)
    sim = Simulator(m)
    sim.setClassicDEVS()
    sim.simulate()
    
    # Get results
    end_simulation = time.time()
    return {
        'time': end_simulation - start_simulation,
        'limit_orders_history': {k: orderbooks_map[k].state.bid_ask_table.get_limit_orders_history() 
                           for k in orderbooks_map},
        'orders_spread': {k: orderbooks_map[k].state.bid_ask_table.get_spread_history() 
                          for k in orderbooks_map},
        'market_orders_history': {k: orderbooks_map[k].state.bid_ask_table.get_market_orders_history() 
                           for k in orderbooks_map},
        'stop_orders_history': {k: orderbooks_map[k].state.bid_ask_table.get_stop_orders_history() 
                           for k in orderbooks_map},
    }

In [3]:
def get_profit(agent):
    profit = 0
    for n_order in agent.strategy.orders_completed.keys():
        o = agent.strategy.orders_completed[n_order]
        # Note: could be different from 'real profit' (from executed price made by exchange)
        profit += (-1) * o.direction.value * o.size * o.price
    return profit

### Experiment

In [4]:
current_time = 0.0
start_time = 0.0
end_time = 100

# Setup contracts for test
contracts = ['INTC', 'INTC']
delay_order, delay_notification = 0, 0

agents, receiver_agents = [], []
agents_delay_map = {}

########
# AGENTS
wakeup_distribution = functools.partial(random.expovariate, 0.5)
n_sellers = 20
n_buyers = 30

# Params price and size
params_buyers_price = [(-5, 4), (-3, 4), (-1, 4)]
params_sellers_price = [(1, 4), (1, 4), (8, 4)]

params_buyers_size = [1]
params_sellers_size = [1]

def get_func_price(delta, sigma):
    def func(t):
        base_price = 30*(1 - (t/30)**1.5 + (t/40)**2.5)
        x = functools.partial(random.normalvariate, base_price + delta, sigma)
        return lambda: x()

    return func

def get_func_size(mu):
    def func(t):
        # x = functools.partial(random.normalvariate, mu, sigma)
#         return lambda: int(x())
        return lambda: 1

    return func

agents_info = []
i = 1
for n_seller in range(n_sellers):
    pp = params_sellers_price[n_seller % len(params_sellers_price)]
    ps = params_sellers_size[n_seller % len(params_sellers_size)]
    agents_info.append(
        {'agent_id': i, 'contract': 'INTC', 'direction': -1,
         'price_distribution': get_func_price(pp[0], pp[1]),
         'size_distribution': get_func_size(ps)})
    i += 1

# Buyers
for n_buyer in range(n_buyers):
    pp = params_buyers_price[n_buyer % len(params_buyers_price)]
    ps = params_buyers_size[n_buyer % len(params_buyers_size)]
    agents_info.append(
        {'agent_id': i, 'contract': 'INTC', 'direction': 1,
         'price_distribution': get_func_price(pp[0], pp[1]),
         'size_distribution': get_func_size(ps)}
    )
    i += 1

# Random agents
for agent_info in agents_info:
    # Initialize parameters
    agent_id = agent_info['agent_id']
    direction = agent_info['direction']
    contract = agent_info['contract']
    price_distribution = agent_info['price_distribution']
    size_distribution = agent_info['size_distribution']
    # In common
    exec_type = 0  # Limit
    cancellation_timeout = 20

    # Initialize agent
    identifier = 'stc_' + str(agent_id)
    new_agent = AgentBase(identifier=identifier, strategy=StochasticStrategy6,
                          strategy_params={'identifier': identifier,
                                           'wakeup_distribution': wakeup_distribution,
                                           'cancellation_timeout': cancellation_timeout,
                                           'direction': direction,
                                           'price_distribution': price_distribution,
                                           'size_distribution': size_distribution,
                                           'initial_cash': 1000, 'max_orders': 100, 
                                           'contract': contract, 'exec_type': exec_type,
                                           'end_time': end_time})
    agents.append(new_agent)
    agents_delay_map[new_agent.identifier] = 0
    
    # All agents receive all information (a lot of messages)
    receiver_agents.append(new_agent)
    
# Impact agent
impact_agent_id = 'impact'
history = pd.read_csv(test_path + 'data/stop_3.csv')
impact_agent = AgentBase(identifier=impact_agent_id, strategy=ReplayVersion3Strategy,
                         strategy_params={
                              'identifier': impact_agent_id, 'history': history
                         })
agents.append(impact_agent)
agents_delay_map[impact_agent.identifier] = 0.0
receiver_agents.append(impact_agent)

# # Spread generator agent
spread_agent_id = 'spread_gen'
spread_agent = AgentBase(identifier=spread_agent_id, strategy=SpreadGeneratorVersion1,
                         strategy_params={
                              'identifier': spread_agent_id, 
                              'size_orders': 1,
                              'delta_price': 5, 'max_orders': 2,
                              'end_time': end_time
                         })
agents.append(spread_agent)
agents_delay_map[spread_agent.identifier] = 0
receiver_agents.append(spread_agent)


#################################
#  SAVE RESULTS
# Simulate
result = simulate(current_time, start_time, end_time, agents, receiver_agents, agents_delay_map, contracts,
              delay_order, delay_notification)

In [5]:
print("Execution time = %f" % result['time'])

# Limit orders
limit_orders = result['limit_orders_history']['INTC']
limit_orders = [(t, [(x[0], x[1]) for x in values for y in range(x[2])]) for t, values in limit_orders]

stop_orders = result['stop_orders_history']['INTC']
stop_orders = [(t, [(x[0], x[1]) for x in values for y in range(x[2])]) for t, values in stop_orders]

# Spreads
spread_list = result['orders_spread']['INTC']
keys_spreads = [x[0] for x in spread_list]
keys_orders = [x[0] for x in limit_orders]
for ks in keys_orders:
    if ks not in keys_spreads:
        spread_list.append((ks, None))


import pickle
with open('snapshot_history.pkl', 'wb') as f:
    to_pickle = {
        'limit_orders': limit_orders,
        'stop_orders': stop_orders,
        'hidden_orders': [],
        'spreads': spread_list
    }
    pickle.dump(to_pickle, f)

Execution time = 40.893631


In [6]:
# len(spread_agent.strategy.orders_completed)

In [7]:
# agents[0].strategy.orders_completed

total_profit = 0
for agent in agents:
    # print("Agent: %s => %f" % (agent.identifier, get_profit(agent)))
    try:
        # print(agent.strategy.direction, agent.strategy.get_remaining_cash())
        total_profit += get_profit(agent)
    except:
        # print('Not available')
        total_profit += get_profit(agent)
print(total_profit)

-9115.099999999999


In [8]:
# agents[-3].strategy.orders_completed

In [9]:
# n_elem=1800
# sell_orders = [y for x in to_pickle['limit_orders'][n_elem:n_elem+1] for y in x[1] if y[0] == 'Sell']
# buy_orders = [y for x in to_pickle['limit_orders'][n_elem:n_elem+1] for y in x[1] if y[0] == 'Buy']

# print(max(x[1] for x in buy_orders), len(buy_orders))
# print(min(x[1] for x in sell_orders), len(sell_orders))

In [10]:
# to_pickle['limit_orders'][9]

In [11]:
# n_elem=1850
# sell_orders = [y for x in to_pickle['limit_orders'][n_elem:n_elem+1] for y in x[1] if y[0] == 'Sell']
# print(min(x[1] for x in sell_orders), len(sell_orders))